In [ ]:
! pip install torchaudio soundfile tensorboard_logger
! sudo apt-get install ffmpeg

In [1]:
import json
import random
import time
import os
import numpy as np
import scipy.signal

import torch
import torch.nn as nn
import torch.optim
import torchaudio
import torch.autograd as autograd
import torch.utils.data as data
from torch.nn import CTCLoss
from torch.autograd import Variable

import tqdm
import soundfile
import tensorboard_logger as tb_log
import librosa 
import IPython

import wave
from math import sin, cos, pi, log, exp, floor, ceil
from torch.nn import BCELoss, BCEWithLogitsLoss , CrossEntropyLoss
import torch.nn.functional as F
from torch.autograd import Variable



__Набор данных для обучения и теста__ состоит из позитивных примеров-аудиодорожек(записано произношение "Alexa") и негативных примеров (любой другой звук/речь/шум).
* Сбор и аугментация позитивных примеров: запись произношения "Alexa" одним человеком в четырёх различных вариациях(две на train, две на test), дублирование этого произношения 160(train), 80(test); изменение скорости (x0.85, x1.15) и добавление шумов (0.01, 0.02, 0.03) - 200 train, 100 test

* Негативные примеры: датасет LibriSpeech (dev-clean) ~(80/20)



Load LibriSpeech dataset as negative speech examples (speak anything but ”Alexa”)

## Загрузка и предобработка данных

In [3]:
dataset_url = 'dev-clean'
libri_data = torchaudio.datasets.LIBRISPEECH('./', url=dataset_url, folder_in_archive='LibriSpeech', download=True)

In [4]:
os.chdir('/content/LibriSpeech/')
# Create small datasets
! mkdir 'aug_wavs' 'train' 'test'
os.mkdir('/content/LibriSpeech/train/1')
os.mkdir('/content/LibriSpeech/train/0')
os.mkdir('/content/LibriSpeech/test/1')
os.mkdir('/content/LibriSpeech/test/0')

In [7]:
rm -r /content/LibriSpeech

In [5]:
# Beta test

! find /content/LibriSpeech/dev-clean/2078 -type f -name '*.flac' -exec mv -i {} /content/LibriSpeech/test/0/  \;
! rm -r /content/LibriSpeech/dev-clean/2078
! find /content/LibriSpeech/dev-clean/2086 -type f -name '*.flac' -exec mv -i {} /content/LibriSpeech/test/0/  \;
! rm -r /content/LibriSpeech/dev-clean/2086
! find /content/LibriSpeech/dev-clean/2277 -type f -name '*.flac' -exec mv -i {} /content/LibriSpeech/test/0/  \;
! rm -r /content/LibriSpeech/dev-clean/2277
! find /content/LibriSpeech/dev-clean/2412 -type f -name '*.flac' -exec mv -i {} /content/LibriSpeech/test/0/  \;
! rm -r /content/LibriSpeech/dev-clean/2412
! find /content/LibriSpeech/dev-clean/2428 -type f -name '*.flac' -exec mv -i {} /content/LibriSpeech/test/0/  \;
! rm -r /content/LibriSpeech/dev-clean/2428
! find /content/LibriSpeech/dev-clean/251 -type f -name '*.flac' -exec mv -i {} /content/LibriSpeech/test/0/  \;
! rm -r /content/LibriSpeech/dev-clean/251

# Beta train
# Move all files in subfolders to another directory without structure
! find /content/LibriSpeech/dev-clean -type f -print0 | xargs -0 mv -t /content/LibriSpeech/train/0

! rm -r /content/LibriSpeech/dev-clean/

## Конвертируем загруженные аудио в .wav формат с помощью shell scrpit 
Load `flac_to_wav.sh`

In [6]:
! chmod +x '/content/LibriSpeech/flac_to_wav.sh' # giving a permision

In [7]:
os.chdir('/content/LibriSpeech/')

In [ ]:
! '/content/LibriSpeech/flac_to_wav.sh'

## Зугружаем .wav c произношением "Alexa" для генерации позитивных примеров
 Load custom wav

In [10]:
os.chdir('/content/')

In [11]:
# C помощью следующих функций аугментируем позитивный .wav 

def add_noise(wav, noise_factor=0.02):
    noise = np.random.normal(size=len(wav))
    noisy_wav = wav + noise_factor * noise
    noisy_wav = noisy_wav.astype(type(wav[0])) # Set initial dtype
    return noisy_wav

def speed_change(wav, speed_factor=1.5):
    y_speed = wav.copy()
    speed_change = speed_factor
    tmp = librosa.effects.time_stretch(y_speed.astype('float64'), speed_change)
    minlen = min(y_speed.shape[0], tmp.shape[0])
    y_speed *= 0 
    y_speed[0:minlen] = tmp[0:minlen]
    return y_speed
    # return librosa.effects.time_stretch(data, speed_factor)

def dummy_data_aug(wav_paths: list, dest_folder, n=10):
    counter = 0 
    for _ in range(n):
        for i in wav_paths:
            y, sr = librosa.load(i)
            y1 = speed_change(y, 1.15)
            librosa.output.write_wav(dest_folder+'%i.wav' % counter, y1, sr)
            counter+=1
            y2 = speed_change(y, 0.85)
            librosa.output.write_wav(dest_folder+'%i.wav' % counter, y2, sr)
            counter+=1
            noisy_y1 = add_noise(y, 0.01)
            librosa.output.write_wav(dest_folder+'%i.wav' % counter, noisy_y1, sr)
            counter+=1
            noisy_y2 = add_noise(y, 0.02)
            librosa.output.write_wav(dest_folder+'%i.wav' % counter, noisy_y2, sr)
            counter+=1
            noisy_y3 = add_noise(y, 0.03)
            librosa.output.write_wav(dest_folder+'%i.wav' % counter, noisy_y3, sr)
            counter+=1
            librosa.output.write_wav(dest_folder+'%i.wav' % counter, y, sr)
            counter+=1
            librosa.output.write_wav(dest_folder+'%i.wav' % counter, y, sr)
            counter+=1
            librosa.output.write_wav(dest_folder+'%i.wav' % counter, y, sr)
            counter+=1
            librosa.output.write_wav(dest_folder+'%i.wav' % counter, y, sr)
            counter+=1

In [12]:
# Positive exapmles to train folder
dummy_data_aug(['/content/Alexa1.wav', '/content/Alexa3.wav'], '/content/LibriSpeech/train/1/', n=20)

In [14]:
# Positive exapmles to test folder
dummy_data_aug(['/content/Alexa2.wav', '/content/Alexa4.wav'], '/content/LibriSpeech/test/1/', n=10)

## Создадим загрузчик данных с извлечением признаков(spectrogram) из .wav 
 Source https://github.com/adiyoss/GCommandsPytorch 

In [3]:
AUDIO_EXTENSIONS = [
    '.wav', '.WAV',
]


def is_audio_file(filename):
    return any(filename.endswith(extension) for extension in AUDIO_EXTENSIONS)


def find_classes(dir):
    classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx


def make_dataset(dir, class_to_idx):
    spects = []
    dir = os.path.expanduser(dir)
    for target in sorted(os.listdir(dir)):
        d = os.path.join(dir, target)
        if not os.path.isdir(d):
            continue

        for root, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                if is_audio_file(fname):
                    path = os.path.join(root, fname)
                    item = (path, class_to_idx[target])
                    spects.append(item)
    return spects


def spect_loader(path, window_size, window_stride, window, normalize, max_len=101):
    y, sr = librosa.load(path, sr=16000)
    # n_fft = 4096
    n_fft = int(sr * window_size)
    win_length = n_fft
    hop_length = int(sr * window_stride)

    # # STFT
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length,
                     win_length=win_length, window=window)
    
    # Compute spectrogram feature
    spect, phase = librosa.magphase(D)


    # S = log(S+1)
    spect = np.log1p(spect) #   

    # spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft) # 128 amplitudes
    

    # make all spects with the same dims
    if spect.shape[1] < max_len:
        pad = np.zeros((spect.shape[0], max_len - spect.shape[1]))
        spect = np.hstack((spect, pad))
    elif spect.shape[1] > max_len:
        spect = spect[:, :max_len]
    spect = np.resize(spect, (1, spect.shape[0], spect.shape[1]))
    spect = torch.FloatTensor(spect)

    # z-score normalization
    if normalize:
        mean = spect.mean()
        std = spect.std()
        if std != 0:
            spect.add_(-mean)
            spect.div_(std)

    return spect


class DatasetLoader(data.Dataset):


    """
    Args:
        root (string): Root directory path.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        window_size: window size for the stft, default value is .02
        window_stride: window stride for the stft, default value is .01
        window_type: typye of window to extract the stft, default value is 'hamming'
        normalize: boolean, whether or not to normalize the spect to have zero mean and one std
        max_len: the maximum length of frames to use
     Attributes:
        classes (list): List of the class names.
        class_to_idx (dict): Dict with items (class_name, class_index).
        spects (list): List of (spects path, class_index) tuples
        STFT parameter: window_size, window_stride, window_type, normalize
    """

    def __init__(self, root, transform=None, target_transform=None, window_size=.02,
                 window_stride=.01, window_type='hann', normalize=True, max_len=101):
        classes, class_to_idx = find_classes(root)
        spects = make_dataset(root, class_to_idx)
        if len(spects) == 0:
            raise (RuntimeError("Found 0 sound files in subfolders of: " + root + "Supported audio file extensions are: " + ",".join(AUDIO_EXTENSIONS)))

        self.root = root
        self.spects = spects
        self.classes = classes
        self.class_to_idx = class_to_idx
        self.transform = transform
        self.target_transform = target_transform
        self.loader = spect_loader
        self.window_size = window_size
        self.window_stride = window_stride
        self.window_type = window_type
        self.normalize = normalize
        self.max_len = max_len

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (spect, target) where target is class_index of the target class.
        """
        path, target = self.spects[index]
          
        spect = self.loader(path, self.window_size, self.window_stride, self.window_type, self.normalize, self.max_len)
        
        if self.transform is not None:
            spect = self.transform(spect)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return spect, target

    def __len__(self):
        return len(self.spects)

## Эмуляция Wake-Word модели на основе highway блоков
![_здесь должна быть картинка архитектуры модели_](https://github.com/eestien/VAD-Attack/blob/master/WWAlexa.png)

 papers __Adversarial Music: Real World Audio Adversary
Against Wake-word Detection System__ _(http://papers.nips.cc/paper/9362-adversarial-music-real-world-audio-adversary-against-wake-word-detection-system.pdf)_, __Adversarial Music: Real world Audio Adversary against Wake-word Detection
Systems
10-708A S19 Final Report__ _(https://sailinglab.github.io/pgm-spring-2019/assets/project/final-reports/project29.pdf)_


In [4]:
class Highway(nn.Module):
    def __init__(self, size, num_layers, f):

        super(Highway, self).__init__()

        # gen features from wav
        # self.spec_layer = Spectrogram.STFT(n_fft=2048, freq_bins=None, hop_length=512, window='hann', freq_scale='no', center=True, pad_mode='reflect', fmin=50,fmax=6000, sr=16000, trainable=False, output_format='Magnitude', device='cuda')
        
        self.num_layers = num_layers
        self.nonlinear = nn.ModuleList([nn.Linear(size, size) for _ in range(num_layers)])
        self.linear = nn.ModuleList([nn.Linear(size, size) for _ in range(num_layers)])
        self.gate = nn.ModuleList([nn.Linear(size, size) for _ in range(num_layers)])
        self.f = f

    def forward(self, x):
        """
            :param x: tensor with shape of [batch_size, size]
            :return: tensor with shape of [batch_size, size]
            applies σ(x) ⨀ (f(G(x))) + (1 - σ(x)) ⨀ (Q(x)) transformation | G and Q is affine transformation,
            f is non-linear transformation, σ(x) is affine transformation with sigmoid non-linearition
            and ⨀ is element-wise multiplication
            """
        # x = self.spec_layer(x)

        for layer in range(self.num_layers):
            gate = torch.sigmoid(self.gate[layer](x))

            nonlinear = self.f(self.nonlinear[layer](x))
            linear = self.linear[layer](x)

            x = gate * nonlinear + (1 - gate) * linear

        return x

# For concatenating features from adjacent frames
def add_context(X_raw, k):
    X_context = []
    for utterance in X_raw:
        n, d = utterance.shape # amplitude values, bottleneck size
        utterance_with_context = torch.zeros((n, (2*k+1)*d)).cuda()
        for j in np.arange(-k, k+1, 1):
            # print(j)
            if j<0:
                utterance_with_context[:j,d*(j+k):d*(j+k+1)] = utterance[-j:]
            elif j==0:
                utterance_with_context[:,d*(j+k):d*(j+k+1)] = utterance
            else:
                utterance_with_context[j:,d*(j+k):d*(j+k+1)] = utterance[:-j]
        X_context.append(utterance_with_context.unsqueeze(0))
    return torch.cat(X_context)

In [5]:
# input size: N_batch x T x Dim.
class myAlexa(nn.Module):
    def __init__(self, n_input, n_bottleneck = 32, n_output= 1, context = 5):
        super(myAlexa, self).__init__()
        self.encoder = Highway(n_input, 4, f=torch.nn.functional.relu)
        self.bottleneck = nn.Linear(n_input, n_bottleneck)
        self.classifier = Highway(n_bottleneck*(2*context+1), 6, f=torch.nn.functional.relu)
        self.linear = nn.Linear(n_bottleneck*(2*context+1), n_output)
        self.context = context
        self.sigmoid = nn.Sigmoid()
       
    def forward(self, x): # x: N_batch x T x Dim.
        # x = torch.FloatTensor(x)
        out = self.encoder(x) # N_batch x T x Dim.
        out = self.bottleneck(out) #  N_batch x T x 32
        out = add_context(out, self.context) # N_batch x T x (32*11)
        out = self.classifier(out) # N x T x (32*11)
        out = self.linear(out) # N x T x 1
        # out = self.sigmoid(out)
        return out # torch.sigmoid(out) # torch.nn.functional.softmax(out, dim=2)# out # torch.FloatTensor(out.cpu()).cuda() # torch.nn.functional.softmax(out, dim=2)
        

        

## Karplus-Strong pytorch nn.Module 
### Генерация гитарной музыкис помощью Karplus-Strong алгоритма

За основную мелодию возьмём набор из нескольких случайных нот.
Зададим параметры, которые будем оптимизировать `boost` и `freq`



  algorithm source https://github.com/mdoege/PySynth/blob/master/demosongs.py

In [6]:
import wave, struct
import numpy as np
from math import sin, cos, pi, log, exp, floor, ceil


## boost & freq init values
boost = 1.1
freq = 32
bpm = 126
transpose = 0


song4_rh = (
  ('e', 8), ('f#', 8),
  ('g*', 4), ('f#', 8), ('e', 8), ('d#*', 4), ('e', 8), ('f#', 8),
  ('b3*', 4), ('c#', 8), ('d#', 8), ('e*', 4), ('d', 8), ('c', 8),
  ('b3*', 4), ('a3', 8), ('g3', 8), ('f#3*', 4), ('g3', 8), ('a3', 8),
  ('b3*', 8), ('a3', 8), ('g3', 8), ('f#3', 8), ('e3*', 4), ('e', 8), ('f#', 8),
  ('g*', 4), ('f#', 8), ('e', 8), ('d#*', 4), ('e', 8), ('f#', 8),
  ('b3*', 4), ('c#', 8), ('d#', 8), ('e*', 4), ('d', 8), ('c', 8),
  ('b3*', 4), ('a3', 8), ('g3', 8), ('g3*', 32), ('f#3*', 32), ('g3*', 32), ('f#3*', 32), ('g3*', 32), ('f#3*', 32), ('g3*', 32), ('f#3*', 6.4), ('g3', 8), ('g3*', -2),
)

pitchhz, keynum = {}, {}

keys_s = ('a', 'a#', 'b',  'c',  'c#', 'd', 'd#', 'e',  'f',  'f#', 'g', 'g#')
keys_f = ('a', 'bb', 'b',  'c',  'db', 'd', 'eb', 'e',  'f',  'gb', 'g', 'ab')
keys_e = ('a', 'bb', 'cb', 'b#', 'db', 'd', 'eb', 'fb', 'e#', 'gb', 'g', 'ab')

def getfreq(pr = False):
	if pr:
		print("Piano key frequencies (for equal temperament):")
		print("Key number\tScientific name\tFrequency (Hz)")
	for k in range(88):
		freq = 27.5 * 2.**(k/12.)
		oct = (k + 9) // 12
		note = '%s%u' % (keys_s[k%12], oct)
		if pr:
			print("%10u\t%15s\t%14.2f" % (k+1, note.upper(), freq))
		pitchhz[note] = freq
		keynum[note] = k
		note = '%s%u' % (keys_f[k%12], oct)
		pitchhz[note] = freq
		keynum[note] = k
		note = '%s%u' % (keys_e[k%12], oct)
		pitchhz[note] = freq
		keynum[note] = k
	return pitchhz, keynum


class AudioGenKS(nn.Module):
    def __init__(self, bpm=bpm, transpose=transpose, pause=0, freq = freq, loud=1, boost=boost,repeat=0,fn="out.wav",silent=False):
        super(AudioGenKS, self).__init__()
        self.bpm = nn.Parameter(torch.autograd.Variable(torch.tensor(bpm, dtype=torch.float)))
        self.transpose = nn.Parameter(torch.autograd.Variable(torch.tensor(transpose, dtype=torch.float)))
        self.pause = pause 
        self.freq = nn.Parameter(torch.autograd.Variable(torch.tensor(freq, dtype=torch.float))) # torch.nn.Parameter(torch.tensor(freq))    
        self.loud = loud # nn.Parameter(torch.autograd.Variable(torch.tensor(loud, dtype=torch.float)))
        self.fn = fn
        self.boost = nn.Parameter(torch.autograd.Variable(torch.tensor(boost, dtype=torch.float)))
        self.repeat = repeat

    def forward(self,song):

        f=wave.open(self.fn,'w')

        f.setnchannels(1)
        f.setsampwidth(2)
        f.setframerate(44100)
        f.setcomptype('NONE','Not Compressed')

        bpmfac = 120./self.bpm
        
        
        pitchhz, keynum = {}, {}

        def getfreq(pr = False):
          if pr:
            print("Piano key frequencies (for equal temperament):")
            print("Key number\tScientific name\tFrequency (Hz)")
          for k in range(88):
            freq = 27.5 * 2.**(k/12.)
            oct = (k + 9) // 12
            note = '%s%u' % (keys_s[k%12], oct)
            if pr:
              print("%10u\t%15s\t%14.2f" % (k+1, note.upper(), freq))
            pitchhz[note] = freq
            keynum[note] = k
            note = '%s%u' % (keys_f[k%12], oct)
            pitchhz[note] = freq
            keynum[note] = k
            note = '%s%u' % (keys_e[k%12], oct)
            pitchhz[note] = freq
            keynum[note] = k
          return pitchhz, keynum

        pitchhz, keynum = getfreq()

        def linint(arr, x):
          "Interpolate an (X, Y) array linearly."
          for v in arr:
            if v[0] == x: return v[1]
          xvals = [v[0] for v in arr]
          ux = max(xvals)
          lx = min(xvals)
          try: assert lx <= x <= ux
          except:
            #print lx, x, ux
            raise
          for v in arr:
            if v[0] > x and v[0] - x <= ux - x:
              ux = v[0]
              uy = v[1]
            if v[0] < x and x - v[0] >= lx - x:
              lx = v[0]
              ly = v[1]		
          #print lx, ly, ux, uy
          return (float(x) - lx) / (ux - lx) * (uy - ly) + ly

        def length(l):
            return 88200./l*bpmfac

        def waves2(hz,l):
            a=44100./hz
            b=float(l)/44100.*hz
            return [a,round(b.item())]

        def asin(x):
            return sin(2.*pi*x)

        def render2(a, b, vol, pos, knum, note, endamp = .25, sm = 10):
          b2 = (1. - self.pause) * b
          l=waves2(a, b2)
          ow=b''
          q=int(l[0]*l[1])

          lf = log(a)
          t = (lf-3.) / (8.5-3.)
          volfac = 1. + .8 * t * cos(pi/5.3*(lf-3.))
          snd_len = int((10.-lf)*q)
          if lf < 4: snd_len *= 2
          x = np.arange(snd_len)
          s = x / float(q)

          ls = np.log(1. + s)
          kp_len = int(l[0])
          kps1 = np.zeros(snd_len)
          kps2 = np.zeros(snd_len)
          kps1[:kp_len] = np.random.normal(size = kp_len)

          for t in range(kp_len):
            kps2[t] = kps1[t:t+sm].mean()
          delt = float(l[0])
          li = int(floor(delt))
          hi = int(ceil(delt))
          ifac = delt % 1
          delt2 = delt * (floor(delt) - 1) / floor(delt)
          ifac2 = delt2 % 1
          falloff = (4./lf*endamp)**(1./l[1])
          for t in range(hi, snd_len):
            v1 = ifac * kps2[t-hi]   + (1.-ifac) * kps2[t-li]
            v2 = ifac2 * kps2[t-hi+1] + (1.-ifac2) * kps2[t-li+1]
            kps2[t] += .5 * (v1 + v2) * falloff
          data[pos:pos+snd_len] += kps2*vol*volfac

        ex_pos = 0.
        t_len = 0
        for y, x in song:
          if x < 0:
            t_len+=length(-2.*x/3.)
          else:
            t_len+=length(x)

        # Empty music
        # data = np.zeros(int((self.repeat+1)*t_len + 20 * 44100))
        data = torch.autograd.Variable(torch.zeros(int((self.repeat+1)*t_len + 20 * 44100)))
        #print len(data)/44100., "s allocated"

        for rp in range(self.repeat+1):
          for nn, x in enumerate(song):
            if not nn % 4:
              print("[%u/%u]\t" % (nn+1,len(song)))
            if x[0]!='r':
              if x[0][-1] == '*':
                vol = self.boost.item()
                note = x[0][:-1]
              else:
                vol = 1.
                note = x[0]
              if not note[-1].isdigit():
                note += '4'		# default to fourth octave
              a=pitchhz[note]
              kn = keynum[note]
              a = a * 2**self.transpose
              if x[1] < 0:
                b=length(-2.*x[1]/3.)
              else:
                b=length(x[1])

              render2(a, b, vol, int(ex_pos), kn, note)
              ex_pos = ex_pos + b

            if x[0]=='r':
              b=length(x[1])
              ex_pos = ex_pos + b

        ##########################################################################
        # Write to output file (in WAV format)
        ##########################################################################
        

        data = data / (data.max() * 2.)
        out_len = int(2. * 44100. + ex_pos+.5)
        data2 = np.zeros(out_len, np.short)
        data2[:] = 32000. * data[:out_len]
        f.writeframes(data2.tostring())
        f.close()
        


## 1. Обучение Wake-word модели

In [36]:
# Define model
model = myAlexa(161) # 161 feature values for each window

# Move model to cuda
model.cuda()


optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = CrossEntropyLoss()

batch_size = 32


# Create train and test dataloaders
train_dataset = DatasetLoader('/content/LibriSpeech/train/', window_size=0.02, window_stride=0.01,
                               window_type='hamming', normalize=True, max_len=101)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, pin_memory=False, sampler=None)

test_dataset = DatasetLoader('/content/LibriSpeech/test/', window_size=0.02, window_stride=0.01,
                               window_type='hamming', normalize=True, max_len=101)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True, pin_memory=False, sampler=None)


# Define train function
def train(loader, model, optimizer, epoch, cuda, log_interval, verbose=True):
    model.train()
 
    global_epoch_loss = 0
    for batch_idx, (data, target) in enumerate(loader):

        # data = torch.add(data, bach) # train sample + generated sample  # THREAT MODEL

        if cuda:
            data, target = data.cuda(), target.cuda()
        try: 
            data = data.reshape(batch_size, -1, 161)
            
            data, target = Variable(data), Variable(target)
            optimizer.zero_grad()
            output = model(data)# .squeeze(1)
            
            loss = criterion(output, target.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            global_epoch_loss += loss.item()
            if verbose:
                if batch_idx % log_interval == 0:
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                            epoch, batch_idx * len(data), len(loader.dataset), 100.
                            * batch_idx / len(loader), loss.item()))
        except Exception:
            print('E')
            pass
    return global_epoch_loss / len(loader.dataset)


def test(loader, model, cuda, verbose=True):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        with torch.no_grad():
            try: 
                data = data.reshape(batch_size, -1, 161)
            
                output = model(data)
                test_loss += criterion(output , target.unsqueeze(-1)).item()
                # test_loss += torch.nn.functional.nll_loss(output, target.unsqueeze(-1)).item()  # sum up batch loss
                pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
                correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            except Exception:
                pass

    test_loss /= len(loader.dataset)
    if verbose:
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(loader.dataset), 100. * correct / len(loader.dataset)))
    return test_loss





In [37]:
def run_train(n_epochs):
    best_valid_loss = 0
    for epoch in range(n_epochs):
        epoch_loss_train = train(train_loader, model, optimizer, epoch, cuda=True, log_interval=25, verbose=True)
        epoch_loss_valid = test(test_loader, model, cuda=True)
        
        epoch += 1

In [38]:
# Run model training
assert torch.cuda.is_available()
run_train(n_epochs=1)

Train Epoch: 0 [0/2641 (0%)]	Loss: 4.615066
Train Epoch: 0 [800/2641 (30%)]	Loss: 0.000372
Train Epoch: 0 [1600/2641 (60%)]	Loss: 0.000274
Train Epoch: 0 [2400/2641 (90%)]	Loss: 0.001860
E

Test set: Average loss: 0.0002, Accuracy: 574/602 (95%)



In [ ]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, '/content/AlexaModel.pt')

### Генерируем базовую струнную мелодию `bach_audio11.wav` с помощью алгоритма Карплуса-Стронга без оптимизированных для атаки параметров. Проверяем насколько эта мелодия понизила точночть wake-word модели

In [ ]:
AG = AudioGenKS(fn='/content/bach_audio11.wav', bpm=126, transpose=1, freq=32, loud=1)

In [ ]:
song4_ = (
  ('e', 6), ('f#', 6), ('e', 6), ('f#', 6), ('e', 6), ('f#', 6), ('e', 6), ('f#', 6))
AG(song4_)

## 1.1 Проверяем точность модели с фоновой музыкой , сгенрерированной без оптимизированных параметров

In [39]:
def test_plus_ks_init(loader, model, cuda, verbose=True):
    model.eval()
    test_loss = 0
    correct = 0
    bach = spect_loader('/content/bach_audio11.wav', 0.02, 0.01, 'hann', True, max_len=101)
    for data, target in loader:

        bach = bach.expand(batch_size,161,101)
        data = torch.add(data, bach) # train sample + generated sample  # 

        if cuda:
            data, target = data.cuda(), target.cuda()
        with torch.no_grad():
            try: 
                data = data.reshape(batch_size, -1, 161)
            
                output = model(data)
                test_loss += criterion(output , target.unsqueeze(-1)).item()
                # test_loss += torch.nn.functional.nll_loss(output, target.unsqueeze(-1)).item()  # sum up batch loss
                pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
                correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            except Exception:
                pass

    test_loss /= len(loader.dataset)
    if verbose:
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(loader.dataset), 100. * correct / len(loader.dataset)))
    return test_loss

In [40]:
def run_test_plus_ks_init(n_epochs):

    for epoch in range(n_epochs):
  
        epoch_loss_valid = test_plus_ks_init(test_loader, model, cuda=True)
        
        epoch += 1

In [41]:
run_test_plus_ks_init(1)


Test set: Average loss: 0.0005, Accuracy: 574/602 (95%)



Точность не упала


## 2. Тренируем аудио-генератор, который является "атакующим"
### Подберем оптимальные параметры для алгоритма генерации музыки

Реализуем защищённый градиентный спуск (projected gradient descent) с следующими ограничениями: (1e-3,1-1e-3). Данные значения были подобраны методом пробы.

In [53]:
# Define criterion 
criterion_att = CrossEntropyLoss()

# Define model
att_model = myAlexa(161) # 128 mel spec values for each window

# Move model to cuda
att_model.cuda()
AG.cuda()

# Define audio gen optimizer
initial_current_params = [p.data.clone() for p in AG.parameters()]
params = list(att_model.parameters()) + initial_current_params
optimizer_att = torch.optim.SGD(params, lr=1e-3)

# Define train function
def train_audio_gen(loader, model, audio_gen, optimizer, criterion, epoch, cuda, log_interval, verbose=True):
    model.eval()
    audio_gen.train()

    # Load basic generated wav
    bach = spect_loader('/content/bach_audio11.wav', 0.02, 0.01, 'hann', True, max_len=101)  
    
    global_epoch_loss = 0
    for batch_idx, (data, target) in enumerate(loader):

        song4_ = (('e', 6), ('f#', 6), ('e', 6), ('f#', 6), ('e', 6), ('f#', 6), ('e', 6), ('f#', 6))
        audio_gen(song4_)
        
        bach = spect_loader('/content/bach_audio11.wav', 0.02, 0.01, 'hann', True, max_len=101)  
        bach = bach.expand(batch_size,161,101)
        data = torch.add(data, bach) # train sample + generated sample 
        
        if cuda:
            data, target = data.cuda(), target.cuda()
        try: 
            data = data.reshape(batch_size, -1, 161)
            
            # data, target = Variable(data), Variable(target)
            optimizer.zero_grad()
            output = model(data)# .squeeze(1)
            
            loss = criterion(output.clamp(1e-3,1-1e-3), target.unsqueeze(-1))  # threshold the parameters and thus doing the projected gradient descent 
            loss = -loss  # flip sign   # THREAT MODEL
            
            loss.backward()
            
            optimizer.step()
            
            global_epoch_loss += loss.item()
            if verbose:
                if batch_idx % log_interval == 0:
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                            epoch, batch_idx * len(data), len(loader.dataset), 100.
                            * batch_idx / len(loader), loss.item()))
        except Exception:
            print('E')
            pass
    return global_epoch_loss / len(loader.dataset)

def test_audio_gen(loader, model, audio_gen, cuda, verbose=True):
    model.eval()
    test_loss = 0
    correct = 0
    audio_gen.eval()
    # Load generated wav
    bach = spect_loader('/content/bach_audio11.wav', 0.02, 0.01, 'hann', True, max_len=101)  # THREAT MODEL
    for data, target in loader:
        bach = bach.expand(batch_size,161,101)
        data = torch.add(data, bach) 
        if cuda:
            data, target = data.cuda(), target.cuda()
        with torch.no_grad():
            try: 
                data = data.reshape(batch_size, -1, 161)
            
                output = model(data)
                test_loss += criterion(output , target.unsqueeze(-1)).item()
                # test_loss += torch.nn.functional.nll_loss(output, target.unsqueeze(-1)).item()  # sum up batch loss
                pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
                correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            except Exception:
                pass

    test_loss /= len(loader.dataset)
    if verbose:
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(loader.dataset), 100. * correct / len(loader.dataset)))
    return test_loss


def run_train_audio_gen(n_epochs=1):
    
    for epoch in range(n_epochs):
        epoch_loss_train = train_audio_gen(train_loader, model=att_model, audio_gen=AG, optimizer=optimizer_ks, criterion=criterion_ks, epoch=epoch, cuda=True, log_interval=25, verbose=True)
        epoch_loss_valid = test_audio_gen(test_loader, model=att_model, audio_gen=AG, cuda=True)
        print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, epoch_loss_train))
        epoch += 1

In [54]:
run_train_audio_gen(n_epochs=1)

Train Epoch: 0 [0/2641 (0%)]	Loss: -8.080856
Train Epoch: 0 [800/2641 (30%)]	Loss: -8.080856
Train Epoch: 0 [1600/2641 (60%)]	Loss: -8.080856
Train Epoch: 0 [2400/2641 (90%)]	Loss: -8.080856
E

Test set: Average loss: 0.2416, Accuracy: 19/602 (3%)

Train Epoch: 0 	Loss: -0.250901


__Вывод :__ Точность сильно понизилась, значит параметры для атакующего аудио-генератора подобраны успешно 

### Генерируем струнную музыку `bach_audio2.wav` c помощью Карплус-Стронг алгоритма с оптимизированными для атаки параметрами  

In [44]:
AG = AudioGenKS(fn='/content/bach_audio2.wav')

In [61]:
song4_ = (
  ('e', 6), ('f#', 6), ('e', 6), ('f#', 6), ('e', 6), ('f#', 6), ('e', 6), ('f#', 6))
AG(song4_)

[1/8]	
[5/8]	
